# Scipy : high-level scientific computing

The **scipy** package contains various toolboxes dedicated to common issues in scientific computing. Its different submodules correspond to different applications, such as interpolation, integration, optimization, image processing, statistics, special functions, etc.

`scipy` is composed of task-specific sub-modules:

| Sub-module	             | Description |
|------------------------|-------------|
| ``scipy.cluster``      | Vector quantization / Kmeans |
| ``scipy.constants``    | Physical and mathematical constants | 
| ``scipy.fftpack``      | Fourier transform | 
| ``scipy.integrate``    | Integration routines | 
| ``scipy.interpolate``  | Interpolation | 
| ``scipy.io``           | Data input and output | 
| ``scipy.linalg``       | Linear algebra routines | 
| ``scipy.ndimage``      | n-dimensional image package | 
| ``scipy.odr``          | Orthogonal distance regression | 
| ``scipy.optimize``     | Optimization | 
| ``scipy.signal``       | Signal processing | 
| ``scipy.sparse``       | Sparse matrices | 
| ``scipy.spatial``      | Spatial data structures and algorithms | 
| ``scipy.special``      | Any special mathematical functions | 
| ``scipy.stats``        | Statistics | 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

### File input/output: `scipy.io`

In [2]:
from scipy import io as spio
a = np.ones((3, 3))
spio.savemat('file.mat', {'a': a}) # savemat expects a dictionary
data = spio.loadmat('file.mat')
data['a']


array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

### Linear algebra operations: ``scipy.linalg``

In [3]:
from scipy import linalg
import numpy as np

#### determinant of a square matrix

In [4]:
arr = np.array([[1, 2],
                [3, 4]])
linalg.det(arr)



-2.0

In [5]:
linalg.det(np.ones((3, 4)))

ValueError: expected square matrix

#### inverse of a square matrix

In [6]:
arr = np.array([[1, 2],
                [3, 4]])
iarr = linalg.inv(arr)
iarr

array([[-2. ,  1. ],
       [ 1.5, -0.5]])

In [ ]:
np.allclose(np.dot(arr, iarr), np.eye(2))

In [ ]:
arr = np.array([[3, 2],
                [6, 4]])
linalg.inv(arr)  

#### singular-value decomposition (SVD) Example

In [ ]:
# singular-value decomposition (SVD)
arr = np.arange(9).reshape((3, 3)) + np.diag([1, 0, 1])
arr

In [ ]:
uarr, spec, vharr = linalg.svd(arr)

In [ ]:
spec

In [ ]:
# The original matrix can be re-composed by matrix multiplication of the outputs of svd with np.dot:
sarr = np.diag(spec)
svd_mat = uarr.dot(sarr).dot(vharr)
svd_mat

In [ ]:
np.allclose(svd_mat, arr)

### Interpolation:  ``scipy.interpolate``

In [ ]:
measured_time = np.linspace(0, 1, 10)
noise = (np.random.random(10)*2 - 1) * 1e-1
measures = np.sin(2 * np.pi * measured_time) + noise

In [ ]:
from scipy.interpolate import interp1d
linear_interp = interp1d(measured_time, measures)

In [ ]:
interpolation_time = np.linspace(0, 1, 50)
linear_results = linear_interp(interpolation_time)

plt.figure(figsize=(6, 4))
plt.plot(measured_time, measures, 'o', ms=6, label='measures')
plt.plot(interpolation_time, linear_results, label='linear interp')
plt.legend()


In [ ]:
cubic_interp = interp1d(measured_time, measures, kind='cubic')
cubic_results = cubic_interp(interpolation_time)

plt.figure(figsize=(6, 4))
plt.plot(measured_time, measures, 'o', ms=6, label='measures')
plt.plot(interpolation_time, linear_results, label='linear interp')
plt.plot(interpolation_time, cubic_results, label='cubic interp')
plt.legend()


### Optimization and fit: ``scipy.optimize``

#### Curve fitting Example

In [ ]:
#First generate some data

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# Seed the random number generator for reproducibility
np.random.seed(0)

x_data = np.linspace(-5, 5, num=50)
y_data = 2.9 * np.sin(1.5 * x_data) + np.random.normal(size=50)

# And plot it

plt.figure(figsize=(6, 4))
plt.scatter(x_data, y_data);


In [ ]:
# Now fit a simple sine function to the data

from scipy import optimize

def test_func(x, a, b):
    return a * np.sin(b * x)

params, params_covariance = optimize.curve_fit(test_func, x_data, y_data,
                                               p0=[2, 2])

print(params)

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(x_data, y_data, label='Data')
plt.plot(x_data, test_func(x_data, params[0], params[1]),
         label='Fitted function')

plt.legend(loc='best');

#### Finding the minimum of a function

In [ ]:
def f(x):
    return x**2 + 10*np.sin(x)


x = np.arange(-10, 10, 0.1)
plt.plot(x, f(x));


In [ ]:
from scipy import optimize

# The default (Nelder Mead)
result = optimize.minimize(f, x0=0)
result

In [ ]:
xmin_global = result.x
result.x # The coordinate of the minimum


In [ ]:
optimize.minimize(f, x0=0, method="L-BFGS-B")  # gradient-descent based methods

In [ ]:
# the function has local minima, the algorithm may find these 
# local minima instead of the global minimum depending on the initial point x0:
res = optimize.minimize(f, x0=3, method="L-BFGS-B")
xmin_local = res.x
res.x

In [ ]:
optimize.basinhopping(f, 0)  

In [ ]:
# Constraints:  interval (0, 10) using the “bounds” argument:
res = optimize.minimize(f, x0=1,
                        bounds=((0, 10), ))
res.x 

#### Finding the roots of a scalar function

In [ ]:
def f(x):
    return x**2 + 10*np.sin(x)

In [ ]:
root = optimize.root(f, x0=1)  # our initial guess is 1
root

In [ ]:
root.x

Note that only one root is found. Inspecting the plot of f reveals that there is a second root around -2.5. We find the exact value of it by adjusting our initial guess:

In [ ]:
root2 = optimize.root(f, x0=-2.5)
root2.x

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)

# Plot the function
ax.plot(x, f(x), 'b-', label="f(x)")

# Plot the minima
xmins = np.array([xmin_global, xmin_local])
ax.plot(xmins, f(xmins), 'go', label="Minima")

# Plot the roots
roots = np.array([root.x, root2.x])
ax.plot(roots, f(roots), 'kv', label="Roots")

# Decorate the figure
ax.legend(loc='best')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.axhline(0, color='gray')


## Statistics and random numbers: scipy.stats

#### Fitting distribution in histogram

In [ ]:
from scipy import stats  
np.random.seed(10)

# create some normal random noisy data
ser = 50*np.random.rand() * np.random.normal(10, 10, 100) + 20

# plot normed histogram
plt.hist(ser, normed=True)

# find minimum and maximum of xticks, so we know
# where we should compute theoretical distribution
xt = plt.xticks()[0]  
xmin, xmax = min(xt), max(xt)  
lnspc = np.linspace(xmin, xmax, len(ser))

# lets try the normal distribution first
m, s = stats.norm.fit(ser) # get mean and standard deviation  
pdf_g = stats.norm.pdf(lnspc, m, s) # now get theoretical values in our interval  
plt.plot(lnspc, pdf_g, label="Norm") # plot it

# exactly same as above
ag,bg,cg = stats.gamma.fit(ser)  
pdf_gamma = stats.gamma.pdf(lnspc, ag, bg,cg)  
plt.plot(lnspc, pdf_gamma, label="Gamma")

# guess what :) 
ab,bb,cb,db = stats.beta.fit(ser)  
pdf_beta = stats.beta.pdf(lnspc, ab, bb,cb, db)  
plt.plot(lnspc, pdf_beta, label="Beta")
plt.legend()

#### Kernel Density Estimation

In [ ]:
from scipy import stats  
np.random.seed(0)


# creating data with two peaks
sampD1 = stats.norm.rvs(loc=-1.0,scale=1,size=300)
sampD2 = stats.norm.rvs(loc=2.0,scale=0.5,size=300)
samp = np.hstack([sampD1,sampD2])

# obtaining the pdf (my_pdf is a function!)
my_pdf = stats.kde.gaussian_kde(samp)

# plotting the result
x = np.linspace(-5,5,100)
plt.plot(x,my_pdf(x),'r') # distribution function
plt.hist(samp,normed=1,alpha=.3) # histogram


#### Statistical tests

In [ ]:
a = np.random.normal(0, 1, size=100)
b = np.random.normal(1, 1, size=10)
stats.ttest_ind(a, b)   


* The T statistic value: it is a number the sign of which is proportional to the difference between the two random processes and the magnitude is related to the significance of this difference.
* the p value: the probability of both processes being identical. If it is close to 1, the two process are almost certainly identical. The closer it is to zero, the more likely it is that the processes have different means.